In [7]:
!pip install --upgrade kaggle

In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("furkanakdeniz/asl-handsign-dataset-grayscaled-thresholded")

print("Path to dataset files:", path)

C:\Users\belho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\belho\.cache\kagglehub\datasets\furkanakdeniz\asl-handsign-dataset-grayscaled-thresholded\versions\1


In [8]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import joblib
import matplotlib.pyplot as plt
import cv2  # Pour redimensionner les images

# Chemin vers le dataset
dataset_path = r"C:\Users\belho\.cache\kagglehub\datasets\furkanakdeniz\asl-handsign-dataset-grayscaled-thresholded\versions\1\asl-dataset\asl-dataset"

# Alphabets disponibles dans le dataset
alphabet_array = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
                  'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

# Charger les images et leurs étiquettes
image_arr = []
image_name_value = []

# Taille cible pour toutes les images
target_size = (28, 28)

for letter in alphabet_array:
    path_letter = os.path.join(dataset_path, "train", letter)
    if not os.path.exists(path_letter):
        print(f"Le dossier {path_letter} n'existe pas.")
        continue

    for image_name in os.listdir(path_letter):
        image_path = os.path.join(path_letter, image_name)
        # Charger et redimensionner l'image
        image = plt.imread(image_path)  # Charger l'image
        if image.ndim == 3:  # Convertir en niveau de gris si l'image est RGB
            image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        resized_image = cv2.resize(image, target_size)  # Redimensionner
        image_arr.append(resized_image.flatten())  # Aplatir et ajouter à la liste
        image_name_value.append(letter)


In [9]:
# Conversion en tableaux numpy
X_data = np.array(image_arr)
y_data = np.array(image_name_value)

# Créer un mapping des étiquettes vers des entiers
label_to_int = {label: idx for idx, label in enumerate(np.unique(y_data))}
y_data = np.array([label_to_int[label] for label in y_data])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)


In [5]:
# Entraîner le modèle SVM
svm_model = SVC(kernel='linear', C=1.0, probability=True)
svm_model.fit(X_train, y_train)

# Évaluer le modèle
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy du modèle SVM: {accuracy:.4f}")
print("\nRapport de classification:\n", classification_report(y_test, y_pred))

# Sauvegarder le modèle entraîné
joblib.dump(svm_model, "svm_hand_sign_model.pkl")
print("Modèle SVM sauvegardé sous 'svm_hand_sign_model.pkl'")

Accuracy du modèle SVM: 0.9419

Rapport de classification:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95       222
           1       0.92      0.96      0.94       248
           2       0.94      0.97      0.96       241
           3       0.94      0.96      0.95       322
           4       0.95      0.97      0.96       297
           5       0.92      0.94      0.93       306
           6       0.95      0.99      0.97       154
           7       0.95      0.99      0.97       257
           8       0.96      0.96      0.96       274
           9       0.94      0.95      0.94       296
          10       0.96      0.97      0.97       255
          11       0.93      0.93      0.93       280
          12       0.92      0.92      0.92       278
          13       0.93      0.91      0.92       173
          14       0.96      0.96      0.96       212
          15       0.87      0.94      0.90       188
          16       0.

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Définir les hyperparamètres à tester
param_grid = {
    'C': [0.1, 1, 10, 100],                # Coefficient de régularisation
    'kernel': ['linear', 'poly', 'rbf'],   # Type de noyau
    'degree': [2, 3, 4],                   # Degré pour le noyau 'poly'
    'gamma': ['scale', 'auto'],            # Coefficient gamma pour 'rbf' et 'poly'
}

# Initialiser le modèle SVM
svm_model = SVC(probability=True)

# Configurer le GridSearch
grid_search = GridSearchCV(
    estimator=svm_model,
    param_grid=param_grid,
    scoring='accuracy',  # Critère pour évaluer les modèles
    cv=3,                # Validation croisée à 3 plis
    verbose=2,           # Afficher la progression
    n_jobs=-1            # Utiliser tous les cœurs disponibles
)

# Lancer la recherche
print("Lancement de la recherche d'hyperparamètres...")
grid_search.fit(X_train, y_train)

# Résultats du GridSearch
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Meilleurs hyperparamètres : {best_params}")
print(f"Meilleure précision moyenne : {best_score:.4f}")

# Évaluer le meilleur modèle sur les données de test
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision sur les données de test : {accuracy:.4f}")
print("\nRapport de classification :\n", classification_report(y_test, y_pred))

# Sauvegarder le meilleur modèle
joblib.dump(best_model, "svm_hand_sign_best_model.pkl")
print("Meilleur modèle SVM sauvegardé sous 'svm_hand_sign_best_model.pkl'")


Lancement de la recherche d'hyperparamètres...
Fitting 3 folds for each of 72 candidates, totalling 216 fits
Meilleurs hyperparamètres : {'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Meilleure précision moyenne : 0.9743
Précision sur les données de test : 0.9849

Rapport de classification :
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       222
           1       0.97      1.00      0.98       248
           2       0.99      1.00      0.99       241
           3       0.99      0.98      0.99       322
           4       0.99      0.99      0.99       297
           5       1.00      0.98      0.99       306
           6       0.99      1.00      0.99       154
           7       0.99      1.00      1.00       257
           8       1.00      0.98      0.99       274
           9       0.98      0.98      0.98       296
          10       1.00      0.98      0.99       255
          11       0.97      0.99      0.98   

In [2]:
import joblib

# Charger le modèle sauvegardé
model_path = "svm_hand_sign_best_model.pkl"  # Chemin du fichier
best_model = joblib.load(model_path)

# Vérifier les classes associées au modèle
if hasattr(best_model, 'classes_'):
    print("Lettres sur lesquelles le modèle a été entraîné :")
    print(best_model.classes_)
else:
    print("Impossible de récupérer les classes. Vérifiez le modèle ou les données.")


Lettres sur lesquelles le modèle a été entraîné :
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [10]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
import joblib

# Charger le meilleur modèle
model_path = "svm_hand_sign_best_model.pkl"
best_model = joblib.load(model_path)

# Vérifier les classes associées au modèle
if hasattr(best_model, 'classes_'):
    print("Lettres sur lesquelles le modèle a été entraîné :")
    print(best_model.classes_)
else:
    print("Impossible de récupérer les classes. Vérifiez le modèle ou les données.")

# Calculer l'AUC pour le meilleur modèle
y_test_bin = label_binarize(y_test, classes=np.arange(len(alphabet_array)))
y_score = best_model.predict_proba(X_test)
auc_score_best = roc_auc_score(y_test_bin, y_score, multi_class="ovr")
print(f"AUC du meilleur modèle : {auc_score_best:.4f}")


Lettres sur lesquelles le modèle a été entraîné :
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
AUC du meilleur modèle : 0.9998
